# **Image Augmentation with AutoEncoder**

In [1]:
import os
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load and preprocess images
def load_images_gdrive(folder_path, target_size=(128, 128)):
    images = []
    filenames = os.listdir(folder_path)
    for filename in filenames:
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img) / 255
        images.append(img)
    return np.array(images)

In [9]:
# Build autoencoder
def build_autoencoder(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)    # make pixel value between 0 and 1 by sigmoid
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder

In [6]:
# Image augmentation
def augment_images(autoencoder, images, save_dir):
    decoded_images = autoencoder.predict(images)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for i, img_array in enumerate(decoded_images):
        img = array_to_img(img_array)
        img.save(os.path.join(save_dir, f'augmented_image_{i}.png'))

In [10]:
# Main code
if __name__ == '__main__':
    # Load original images
    folder_path = '/content/drive/MyDrive/Samples/crawling/b_bicycle/'
    images = load_images_gdrive(folder_path)

    # Fit autoencoder
    autoencoder = build_autoencoder(input_shape=(128, 128, 3))
    autoencoder.fit(images, images, epochs=20, batch_size=20)

    # Save augmented images
    save_dir = '/content/drive/MyDrive/Augmentation/Crawling/b_bicycle'
    augment_images(autoencoder, images, save_dir)

Epoch 1/20
9/9 [==============================] - 11s 1s/step - loss: 0.6863
Epoch 2/20
9/9 [==============================] - 8s 879ms/step - loss: 0.6513
Epoch 3/20
9/9 [==============================] - 10s 1s/step - loss: 0.6163
Epoch 4/20
9/9 [==============================] - 11s 1s/step - loss: 0.5928
Epoch 5/20
9/9 [==============================] - 7s 814ms/step - loss: 0.5786
Epoch 6/20
9/9 [==============================] - 12s 1s/step - loss: 0.5693
Epoch 7/20
9/9 [==============================] - 8s 828ms/step - loss: 0.5646
Epoch 8/20
9/9 [==============================] - 9s 1s/step - loss: 0.5630
Epoch 9/20
9/9 [==============================] - 9s 992ms/step - loss: 0.5619
Epoch 10/20
9/9 [==============================] - 13s 1s/step - loss: 0.5589
Epoch 11/20
9/9 [==============================] - 10s 1s/step - loss: 0.5579
Epoch 12/20
9/9 [==============================] - 8s 893ms/step - loss: 0.5569
Epoch 13/20
9/9 [==============================] - 10s 1s/step -